### **Web Scraping:** 
**El web scraping es el proceso de extracción automatizada de datos de sitios web. Utiliza programas de software para recorrer páginas web, analizar su estructura y extraer información específica de ellas. Este proceso puede incluir la descarga de contenido web, el análisis de HTML para identificar patrones de datos y la extracción de datos relevantes según ciertos criterios, como etiquetas HTML específicas, clases CSS o ubicaciones en la página. El objetivo del web scraping es obtener datos de manera eficiente y automatizada para su posterior análisis, almacenamiento o uso en otras aplicaciones.**

#### Importamos librerias necesarias para nuestro trabajo

In [2]:
import pandas as pd  # Importar la librería pandas para manipulación y análisis de datos
import os  # Importar el módulo os para operaciones relacionadas con el sistema operativo
import re  # Importar el módulo re para operaciones de expresiones regulares
import requests  # Importar el módulo requests para realizar solicitudes HTTP
from bs4 import BeautifulSoup  # Importar BeautifulSoup desde el módulo bs4 para analizar HTML
import fastparquet  # Importar la biblioteca fastparquet para leer y escribir archivos Parquet

#### Primer WebScraping: página web de **Comisión de Taxis y Limusinas de Nueva York (TLC)**

Registros de viajes en taxis amarillos y verdes, incluye información sobre fechas, horas, ubicaciones, distancias, tarifas y tipos de pago. Estos datos fueron recopilados por proveedores tecnológicos autorizados y presentados a la Comisión de Taxis y Limusinas de Nueva York (TLC). https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [2]:
def create_directory(directory):
    """Crea un directorio si no existe."""
    if not os.path.exists(directory):  # Verificar si el directorio no existe
        os.makedirs(directory)  # Crear el directorio si no existe

def download_file(url, filename):
    """Descarga un archivo desde la URL y lo guarda con el nombre especificado."""
    response = requests.get(url)  # Realizar una solicitud GET a la URL especificada
    if response.status_code == 200:  # Verificar si la solicitud fue exitosa (código de estado 200)
        with open(filename, 'wb') as file:  # Abrir el archivo en modo binario para escritura
            file.write(response.content)  # Escribir el contenido de la respuesta en el archivo
        print(f"Archivo descargado: {filename}")  # Imprimir un mensaje indicando que el archivo ha sido descargado

# Crear carpeta de destino si no existe
carpeta_destino = "../0-DataSets/1-DatosScraping"
create_directory(carpeta_destino)

# Obtener la lista de archivos en la carpeta de destino
archivos_locales = os.listdir(carpeta_destino)

# Generar enlaces para los meses del 01 al 12
meses = []
for nums in range(1,13):  # Iterar sobre números del 1 al 12
    if nums < 10:
        meses.append('0' + str(nums))  # Agregar un cero delante si el número es menor que 10
    else:
        meses.append(str(nums))  # Convertir el número a cadena y agregarlo sin modificar si es mayor o igual a 10

# URL de la página web
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

# Realizar la solicitud HTTP a la página
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Parsear el contenido HTML de la página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrar todos los enlaces que contienen 'yellow' o 'green' y tienen el formato esperado para 2023
    links_pagina = {a['href'] for a in soup.find_all('a', href=True) if re.match(r'^.*(yellow|green)_tripdata_(2023|2024)-\d{2}\.parquet$', a['href'])}

    # Combinar enlaces generados dinámicamente con los obtenidos de la página
    links_totales = links_pagina.copy()  # Crear una copia para mantener los enlaces originales

    links_totales.update({
        f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{año}-{mes}.parquet" 
        for año in ['2023', '2024']  # Iterar sobre los años
        for mes in meses
    })
    links_totales.update({
        f"https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_{año}-{mes}.parquet" 
        for año in ['2023', '2024']  # Iterar sobre los años
        for mes in meses
    })

    # Realizar la descarga de los archivos
    for link in sorted(links_totales):  # Iterar sobre los enlaces ordenados alfabéticamente
        nombre_archivo = link.split("/")[-1]  # Obtener el nombre del archivo desde la URL
        archivo_destino = os.path.join(carpeta_destino, nombre_archivo)  # Construir la ruta completa del archivo destino

        if nombre_archivo not in archivos_locales:  # Verificar si el archivo no existe localmente
            download_file(link, archivo_destino)  # Descargar el archivo desde el enlace proporcionado
        else:
            print(f"El archivo {nombre_archivo} ya existe localmente.")  # Imprimir un mensaje indicando que el archivo ya existe
else:
    print(f"No se pudo acceder a la página. Código de estado: {response.status_code}")  # Imprimir un mensaje de error si la solicitud no fue exitosa


Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-01.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-02.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-03.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-04.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-05.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-06.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-07.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-08.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-09.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-10.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-11.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-12.parquet
Archivo descargado: ../0-Dat

#### Realizamos un `ETL` de los `archivos descargados` para tener un Dataset según proposito del proyecto.

###### **`DICCIONARIO`** <br>* 'VendorID': Un código que indica el proveedor de LPEP que proporcionó el registro.<br>1= Tecnologías móviles creativas, LLC;<br> 2= ​​VeriFone Inc.<br>* 'lpep_pickup_datetime': La fecha y hora en que se activó el medidor.<br>* 'lpep_dropoff_datetime': La fecha y hora en que se desconectó el medidor.<br>* 'store_and_fwd_flag': Esta bandera indica si el registro de viaje se mantuvo en el vehículo.memoria antes de enviar al proveedor, también conocido como "almacenar y reenviar",porque el vehículo no tenía conexión con el servidor.<br>Y = viaje de almacenamiento y avance<br>N= no es un viaje de ida y vuelta<br>* 'RatecodeID': El código de tarifa final vigente al finalizar el viaje.<br>1= Tarifa estándar<br>2=JFK<br>3=Newark<br>4=Nasáu o Westchester<br>5=Tarifa negociada<br>6=viaje en grupo<br>* 'PULocationID': Zona de Taxi TLC en la que estaba activado el taxímetro<br>* 'DOLocationID': Zona de Taxi TLC en la que se desactivó el taxímetro<br>* 'passenger_count': El número de pasajeros en el vehículo. Este es un valor ingresado por el conductor.<br>* 'trip_distance': La distancia del viaje transcurrido en millas reportadas por el taxímetro.<br>* 'fare_amount': La tarifa de tiempo y distancia calculada por el taxímetro.<br>* 'extra': Extras y recargos varios. Actualmente esto sólo incluye los cargos de $0,50 y $1 por hora pico y por noche.<br>* 'mta_tax': Impuesto MTA de $0.50 que se activa automáticamente según el medidor tasa en uso.<br>* 'tip_amount': Monto de la propina: este campo se completa automáticamente para tarjetas de crédito. consejos. Las propinas en efectivo no están incluidas.<br>* 'tolls_amount': Importe total de todos los peajes pagados en el viaje.<br>* 'ehail_fee': ???<br>* 'improvement_surcharge': Recargo por mejora de $0.30 aplicado en viajes granizados en la bandera gota. El recargo por mejora comenzó a cobrarse en 2015.<br>* 'total_amount': El importe total cobrado a los pasajeros. No incluye propinas en efectivo.<br>* 'payment_type': Un código numérico que indica cómo el pasajero pagó el viaje.<br>1= tarjeta de crédito<br>2= ​​Efectivo<br>3= Sin cargo<br>4= Disputa<br>5= Desconocido<br>6= Viaje anulado<br>* 'trip_type': Un código que indica si el viaje fue un llamado callejero o un despacho.que se asigna automáticamente según la tarifa medida en uso, pero puede ser modificado por el conductor.<br>1= Granizo callejero<br>2= ​​Despacho<br>* 'congestion_surcharge':"


In [3]:
# Buscamos los archivos Parquet recién guardados en la Carpeta BBDDscrapeados
carpeta = "../0-DataSets/1-DatosScraping"

# Obtener la lista de archivos Parquet en la carpeta
archivos = os.listdir(carpeta)

# Filtrar los archivos que contienen la palabra "green" en su nombre
archivos_green = [archivo for archivo in archivos if "green" in archivo]

# Crear una lista para almacenar los DataFrames
dataframes_green = []

# Leer y concatenar los archivos Parquet que contienen "green" en su nombre
for archivo in archivos_green:
    ruta_archivo = os.path.join(carpeta, archivo)  # Obtener la ruta completa del archivo
    df = pd.read_parquet(ruta_archivo, engine='fastparquet')  # Leer el archivo Parquet y cargarlo como DataFrame
    dataframes_green.append(df)  # Agregar el DataFrame a la lista

# Concatenar todos los DataFrames en uno solo
green_concatenado = pd.concat(dataframes_green, ignore_index=True)

# Mostrar el DataFrame concatenado
green_concatenado


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2023-01-23 13:23:52,2023-01-23 13:41:00,N,1.0,43.0,229.0,1.0,2.57,17.7,0.0,0.5,3.29,0.00,NaN,1.0,25.24,1.0,1.0,2.75
1,2.0,2023-01-06 10:38:28,2023-01-06 11:08:31,N,1.0,166.0,92.0,1.0,10.71,46.4,0.0,0.5,0.00,6.55,NaN,1.0,54.45,1.0,1.0,0.00
2,2.0,2023-01-26 07:17:34,2023-01-26 07:24:02,N,1.0,74.0,41.0,1.0,0.79,7.9,0.0,0.5,1.88,0.00,NaN,1.0,11.28,1.0,1.0,0.00
3,2.0,2023-01-18 12:36:50,2023-01-18 12:46:36,N,1.0,74.0,166.0,1.0,1.25,9.3,0.0,0.5,0.00,0.00,NaN,1.0,10.80,2.0,1.0,0.00
4,2.0,2023-01-08 02:15:30,2023-01-08 02:26:48,N,1.0,47.0,174.0,2.0,2.48,14.2,1.0,0.5,0.00,0.00,NaN,1.0,16.70,2.0,1.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168712,2.0,2024-01-29 17:12:44,2024-01-29 17:22:25,N,1.0,43.0,143.0,1.0,2.23,12.8,2.5,0.5,2.00,0.00,NaN,1.0,21.55,1.0,1.0,2.75
168713,2.0,2024-01-20 17:15:58,2024-01-20 17:29:02,N,1.0,97.0,97.0,1.0,1.59,13.5,0.0,0.5,3.00,0.00,NaN,1.0,18.00,1.0,1.0,0.00
168714,2.0,2024-01-29 21:42:23,2024-01-29 21:47:39,N,1.0,83.0,83.0,1.0,0.84,6.5,1.0,0.5,3.00,0.00,NaN,1.0,12.00,1.0,1.0,0.00
168715,1.0,2024-01-17 08:10:40,2024-01-17 08:17:37,N,1.0,74.0,74.0,1.0,0.70,7.2,0.0,1.5,0.00,0.00,NaN,1.0,8.70,1.0,1.0,0.00


In [4]:
# Agregar una nueva columna 'total_pay' al DataFrame green_concatenado que contiene la suma de 'tip_amount' y 'total_amount'
green_concatenado['amount'] = green_concatenado['tip_amount'] + green_concatenado['total_amount']

# Crear un nuevo DataFrame 'greendf' con las columnas seleccionadas
greendf = green_concatenado[['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'RatecodeID', 'passenger_count', 'trip_distance','amount']]

# Mostrar el DataFrame 'greendf'
greendf

,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,RatecodeID,passenger_count,trip_distance,amount
0,2023-01-23 13:23:52,2023-01-23 13:41:00,43.0,229.0,1.0,1.0,2.57,28.53
1,2023-01-06 10:38:28,2023-01-06 11:08:31,166.0,92.0,1.0,1.0,10.71,54.45
2,2023-01-26 07:17:34,2023-01-26 07:24:02,74.0,41.0,1.0,1.0,0.79,13.16
3,2023-01-18 12:36:50,2023-01-18 12:46:36,74.0,166.0,1.0,1.0,1.25,10.80
4,2023-01-08 02:15:30,2023-01-08 02:26:48,47.0,174.0,1.0,2.0,2.48,16.70
...,...,...,...,...,...,...,...,...
168712,2024-01-29 17:12:44,2024-01-29 17:22:25,43.0,143.0,1.0,1.0,2.23,23.55
168713,2024-01-20 17:15:58,2024-01-20 17:29:02,97.0,97.0,1.0,1.0,1.59,21.00
168714,2024-01-29 21:42:23,2024-01-29 21:47:39,83.0,83.0,1.0,1.0,0.84,15.00
168715,2024-01-17 08:10:40,2024-01-17 08:17:37,74.0,74.0,1.0,1.0,0.70,8.70


In [5]:
# Renombramos las columnas
greendf.rename(columns={'lpep_pickup_datetime': 'pickup_datetime','lpep_dropoff_datetime': 'dropoff_datetime'}, inplace=True)

# Informacion del DataFrame
greendf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168717 entries, 0 to 168716
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   pickup_datetime   168717 non-null  datetime64[ns]
 1   dropoff_datetime  168717 non-null  datetime64[ns]
 2   PULocationID      168717 non-null  float64       
 3   DOLocationID      168717 non-null  float64       
 4   RatecodeID        156836 non-null  float64       
 5   passenger_count   156836 non-null  float64       
 6   trip_distance     168717 non-null  float64       
 7   amount            168717 non-null  float64       
dtypes: datetime64[ns](2), float64(6)
memory usage: 10.3 MB


C:\Users\Pc\AppData\Local\Temp\ipykernel_8356\3680562346.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  greendf.rename(columns={'lpep_pickup_datetime': 'pickup_datetime','lpep_dropoff_datetime': 'dropoff_datetime'}, inplace=True)


In [6]:
greendf =greendf.dropna() # Eliminamos solo el 7% de las filas donde contiene valores NaN
greendf

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,RatecodeID,passenger_count,trip_distance,amount
0,2023-01-23 13:23:52,2023-01-23 13:41:00,43.0,229.0,1.0,1.0,2.57,28.53
1,2023-01-06 10:38:28,2023-01-06 11:08:31,166.0,92.0,1.0,1.0,10.71,54.45
2,2023-01-26 07:17:34,2023-01-26 07:24:02,74.0,41.0,1.0,1.0,0.79,13.16
3,2023-01-18 12:36:50,2023-01-18 12:46:36,74.0,166.0,1.0,1.0,1.25,10.80
4,2023-01-08 02:15:30,2023-01-08 02:26:48,47.0,174.0,1.0,2.0,2.48,16.70
...,...,...,...,...,...,...,...,...
168712,2024-01-29 17:12:44,2024-01-29 17:22:25,43.0,143.0,1.0,1.0,2.23,23.55
168713,2024-01-20 17:15:58,2024-01-20 17:29:02,97.0,97.0,1.0,1.0,1.59,21.00
168714,2024-01-29 21:42:23,2024-01-29 21:47:39,83.0,83.0,1.0,1.0,0.84,15.00
168715,2024-01-17 08:10:40,2024-01-17 08:17:37,74.0,74.0,1.0,1.0,0.70,8.70


In [7]:
greendf.RatecodeID.unique()

array([ 1.,  5.,  3.,  2.,  4., 99.,  6.])

In [8]:
greendf = greendf[greendf["passenger_count"] != 99.]#Eliminamos los que contienen valores 0 pasajeros por que no es una carrera
len(greendf)

156836

In [9]:
greendf.passenger_count.unique()

array([1., 2., 3., 6., 5., 0., 4., 8., 7., 9.])

In [10]:
greendf = greendf[greendf["passenger_count"] != 0.]#Eliminamos los que contienen valores 0 pasajeros por que no es una carrera
len(greendf)

155512

In [11]:
len(greendf[greendf["trip_distance"] == 0.])

7958

In [12]:
greendf = greendf[greendf["trip_distance"] != 0.]#Eliminamos los que contienen valores 0 pasajeros por que no es una carrera
len(greendf)

147554

In [13]:
# Creamos una columna llamada service_type con el valor 'green'
greendf['service_type'] = 'green'
greendf

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,RatecodeID,passenger_count,trip_distance,amount,service_type
0,2023-01-23 13:23:52,2023-01-23 13:41:00,43.0,229.0,1.0,1.0,2.57,28.53,green
1,2023-01-06 10:38:28,2023-01-06 11:08:31,166.0,92.0,1.0,1.0,10.71,54.45,green
2,2023-01-26 07:17:34,2023-01-26 07:24:02,74.0,41.0,1.0,1.0,0.79,13.16,green
3,2023-01-18 12:36:50,2023-01-18 12:46:36,74.0,166.0,1.0,1.0,1.25,10.80,green
4,2023-01-08 02:15:30,2023-01-08 02:26:48,47.0,174.0,1.0,2.0,2.48,16.70,green
...,...,...,...,...,...,...,...,...,...
168712,2024-01-29 17:12:44,2024-01-29 17:22:25,43.0,143.0,1.0,1.0,2.23,23.55,green
168713,2024-01-20 17:15:58,2024-01-20 17:29:02,97.0,97.0,1.0,1.0,1.59,21.00,green
168714,2024-01-29 21:42:23,2024-01-29 21:47:39,83.0,83.0,1.0,1.0,0.84,15.00,green
168715,2024-01-17 08:10:40,2024-01-17 08:17:37,74.0,74.0,1.0,1.0,0.70,8.70,green


In [14]:
porcentaje = 1#0.04
filas_totales = len(greendf)
filas_a_tomar = int(filas_totales*porcentaje)
filas_green_seleccionadas = greendf.sample(n=filas_a_tomar) 
filas_green_seleccionadas

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,RatecodeID,passenger_count,trip_distance,amount,service_type
133303,2023-11-07 17:32:58,2023-11-07 17:44:51,75.0,42.0,1.0,6.0,2.08,15.00,green
8598,2023-01-09 10:14:18,2023-01-09 10:23:18,243.0,247.0,1.0,1.0,1.38,12.20,green
105378,2023-08-19 00:01:16,2023-08-19 00:10:17,95.0,95.0,1.0,1.0,1.62,12.50,green
113160,2023-09-15 12:37:38,2023-09-15 12:59:15,129.0,82.0,1.0,1.0,1.64,27.86,green
1266,2023-01-18 07:32:50,2023-01-18 07:40:12,75.0,236.0,1.0,2.0,1.39,18.97,green
...,...,...,...,...,...,...,...,...,...
97775,2023-08-13 08:44:00,2023-08-13 08:50:11,129.0,129.0,1.0,1.0,2.05,19.34,green
57736,2023-05-28 13:06:28,2023-05-28 13:18:42,74.0,151.0,1.0,1.0,2.20,24.00,green
139336,2023-11-21 19:05:28,2023-11-21 19:09:29,75.0,262.0,1.0,1.0,0.80,13.25,green
131851,2023-11-20 14:57:58,2023-11-20 15:07:13,75.0,41.0,1.0,2.0,0.97,11.50,green


#### Llamamos a los archivos de Taxis Yellow

In [15]:
# Carpeta donde se encuentran los archivos Parquet
carpeta = "../0-DataSets/1-DatosScraping"

# Obtener la lista de archivos Parquet en la carpeta
archivos = os.listdir(carpeta)

# Filtrar los archivos que contienen la palabra "yellow" en su nombre
archivos_yellow = [archivo for archivo in archivos if "yellow" in archivo]

# Crear una lista para almacenar los DataFrames
dataframes_yellow = []

# Leer y concatenar los archivos Parquet que contienen "yellow" en su nombre
for archivo in archivos_yellow:
    ruta_archivo = os.path.join(carpeta, archivo)  # Obtener la ruta completa del archivo
    df = pd.read_parquet(ruta_archivo, engine='fastparquet')  # Leer el archivo Parquet y cargarlo como DataFrame
    dataframes_yellow.append(df)  # Agregar el DataFrame a la lista

# Concatenar todos los DataFrames en uno solo
yellow_concatenado = pd.concat(dataframes_yellow, ignore_index=True)

# Mostrar el DataFrame concatenado
yellow_concatenado


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Airport_fee
0,1.0,2023-01-19 07:38:54,2023-01-19 07:58:14,NaN,2.70,NaN,None,211.0,162.0,0.0,17.0,0.0,0.5,4.20,0.00,1.0,25.20,NaN,NaN,NaN
1,1.0,2023-01-01 16:37:02,2023-01-01 16:47:54,0.0,2.00,1.0,N,142.0,238.0,1.0,13.5,2.5,0.5,3.50,0.00,1.0,21.00,2.5,0.0,NaN
2,2.0,2023-01-12 22:06:35,2023-01-12 22:22:45,1.0,2.64,1.0,N,186.0,140.0,1.0,17.0,1.0,0.5,4.40,0.00,1.0,26.40,2.5,0.0,NaN
3,1.0,2023-01-27 13:12:07,2023-01-27 13:28:26,2.0,1.30,1.0,N,162.0,164.0,1.0,12.8,2.5,0.5,3.35,0.00,1.0,20.15,2.5,0.0,NaN
4,1.0,2023-01-05 15:17:45,2023-01-05 15:22:41,1.0,1.00,1.0,N,249.0,231.0,1.0,7.2,2.5,0.5,2.20,0.00,1.0,13.40,2.5,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8254960,2.0,2024-01-12 11:49:04,2024-01-12 12:49:18,1.0,20.18,2.0,N,237.0,132.0,1.0,70.0,0.0,0.5,24.28,6.94,1.0,105.22,2.5,NaN,0.0
8254961,2.0,2024-01-11 18:28:55,2024-01-11 18:42:23,1.0,1.01,1.0,N,230.0,233.0,1.0,12.8,2.5,0.5,3.86,0.00,1.0,23.16,2.5,NaN,0.0
8254962,2.0,2024-01-12 15:08:25,2024-01-12 15:14:14,1.0,0.59,1.0,N,237.0,141.0,1.0,7.2,2.5,0.5,4.11,0.00,1.0,17.81,2.5,NaN,0.0
8254963,2.0,2024-01-16 18:26:37,2024-01-16 18:36:26,1.0,1.17,1.0,N,186.0,48.0,1.0,10.0,2.5,0.5,4.12,0.00,1.0,20.62,2.5,NaN,0.0


In [16]:
# Agregar una nueva columna 'total_pay' al DataFrame yellow_concatenado que contiene la suma de 'tip_amount' y 'total_amount'
yellow_concatenado['amount'] = yellow_concatenado['tip_amount'] + yellow_concatenado['total_amount']

# Crear un nuevo DataFrame 'yellowdf' con las columnas seleccionadas
yellowdf = yellow_concatenado[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'RatecodeID', 'passenger_count', 'trip_distance','amount']]

# Mostrar el DataFrame 'yellowdf'
yellowdf

,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID,RatecodeID,passenger_count,trip_distance,amount
0,2023-01-19 07:38:54,2023-01-19 07:58:14,211.0,162.0,NaN,NaN,2.70,29.40
1,2023-01-01 16:37:02,2023-01-01 16:47:54,142.0,238.0,1.0,0.0,2.00,24.50
2,2023-01-12 22:06:35,2023-01-12 22:22:45,186.0,140.0,1.0,1.0,2.64,30.80
3,2023-01-27 13:12:07,2023-01-27 13:28:26,162.0,164.0,1.0,2.0,1.30,23.50
4,2023-01-05 15:17:45,2023-01-05 15:22:41,249.0,231.0,1.0,1.0,1.00,15.60
...,...,...,...,...,...,...,...,...
8254960,2024-01-12 11:49:04,2024-01-12 12:49:18,237.0,132.0,2.0,1.0,20.18,129.50
8254961,2024-01-11 18:28:55,2024-01-11 18:42:23,230.0,233.0,1.0,1.0,1.01,27.02
8254962,2024-01-12 15:08:25,2024-01-12 15:14:14,237.0,141.0,1.0,1.0,0.59,21.92
8254963,2024-01-16 18:26:37,2024-01-16 18:36:26,186.0,48.0,1.0,1.0,1.17,24.74


In [17]:
# Renombramos las columnas
yellowdf.rename(columns={'tpep_pickup_datetime': 'pickup_datetime','tpep_dropoff_datetime': 'dropoff_datetime'}, inplace=True)

# Informacion del DataFrame
yellowdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8254965 entries, 0 to 8254964
Data columns (total 8 columns):
 #   Column            Dtype         
---  ------            -----         
 0   pickup_datetime   datetime64[ns]
 1   dropoff_datetime  datetime64[ns]
 2   PULocationID      float64       
 3   DOLocationID      float64       
 4   RatecodeID        float64       
 5   passenger_count   float64       
 6   trip_distance     float64       
 7   amount            float64       
dtypes: datetime64[ns](2), float64(6)
memory usage: 503.8 MB


C:\Users\Pc\AppData\Local\Temp\ipykernel_8356\2899327453.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yellowdf.rename(columns={'tpep_pickup_datetime': 'pickup_datetime','tpep_dropoff_datetime': 'dropoff_datetime'}, inplace=True)


In [18]:
yellowdf =yellowdf.dropna() # Eliminamos solo el 7% de las filas donde contiene valores NaN
yellowdf

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,RatecodeID,passenger_count,trip_distance,amount
1,2023-01-01 16:37:02,2023-01-01 16:47:54,142.0,238.0,1.0,0.0,2.00,24.50
2,2023-01-12 22:06:35,2023-01-12 22:22:45,186.0,140.0,1.0,1.0,2.64,30.80
3,2023-01-27 13:12:07,2023-01-27 13:28:26,162.0,164.0,1.0,2.0,1.30,23.50
4,2023-01-05 15:17:45,2023-01-05 15:22:41,249.0,231.0,1.0,1.0,1.00,15.60
5,2023-01-05 01:01:22,2023-01-05 01:16:29,186.0,87.0,1.0,1.0,4.07,34.72
...,...,...,...,...,...,...,...,...
8254960,2024-01-12 11:49:04,2024-01-12 12:49:18,237.0,132.0,2.0,1.0,20.18,129.50
8254961,2024-01-11 18:28:55,2024-01-11 18:42:23,230.0,233.0,1.0,1.0,1.01,27.02
8254962,2024-01-12 15:08:25,2024-01-12 15:14:14,237.0,141.0,1.0,1.0,0.59,21.92
8254963,2024-01-16 18:26:37,2024-01-16 18:36:26,186.0,48.0,1.0,1.0,1.17,24.74


In [19]:
yellowdf.RatecodeID.unique()

array([ 1.,  3.,  2.,  5.,  4., 99.,  6.])

In [20]:
yellowdf = yellowdf[yellowdf["passenger_count"] != 99.]#Eliminamos los que contienen valores 0 pasajeros por que no es una carrera
len(yellowdf)

7964581

In [21]:
yellowdf.passenger_count.unique()

array([0., 1., 2., 4., 3., 6., 5., 8., 7., 9.])

In [22]:
yellowdf = yellowdf[yellowdf["passenger_count"] != 0.]#Eliminamos los que contienen valores 0 pasajeros por que no es una carrera
len(yellowdf)

7841410

In [23]:
len(yellowdf[yellowdf["trip_distance"] == 0.])

100609

In [24]:
yellowdf = yellowdf[yellowdf["trip_distance"] != 0.]#Eliminamos los que contienen valores 0 pasajeros por que no es una carrera
len(yellowdf)

7740801

In [25]:
# Creamos una columna llamada service_type con el valor 'yellow'
yellowdf['service_type'] = 'yellow'
yellowdf

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,RatecodeID,passenger_count,trip_distance,amount,service_type
2,2023-01-12 22:06:35,2023-01-12 22:22:45,186.0,140.0,1.0,1.0,2.64,30.80,yellow
3,2023-01-27 13:12:07,2023-01-27 13:28:26,162.0,164.0,1.0,2.0,1.30,23.50,yellow
4,2023-01-05 15:17:45,2023-01-05 15:22:41,249.0,231.0,1.0,1.0,1.00,15.60,yellow
5,2023-01-05 01:01:22,2023-01-05 01:16:29,186.0,87.0,1.0,1.0,4.07,34.72,yellow
6,2023-01-14 01:46:12,2023-01-14 02:05:00,249.0,262.0,1.0,2.0,4.86,35.30,yellow
...,...,...,...,...,...,...,...,...,...
8254960,2024-01-12 11:49:04,2024-01-12 12:49:18,237.0,132.0,2.0,1.0,20.18,129.50,yellow
8254961,2024-01-11 18:28:55,2024-01-11 18:42:23,230.0,233.0,1.0,1.0,1.01,27.02,yellow
8254962,2024-01-12 15:08:25,2024-01-12 15:14:14,237.0,141.0,1.0,1.0,0.59,21.92,yellow
8254963,2024-01-16 18:26:37,2024-01-16 18:36:26,186.0,48.0,1.0,1.0,1.17,24.74,yellow


In [26]:
porcentaje = 1#0.04
filas_totalesy = len(yellowdf)
filas_a_tomary = int(filas_totalesy*porcentaje)
filas_yellow_seleccionadas = yellowdf.sample(n=filas_a_tomary) 
filas_yellow_seleccionadas

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,RatecodeID,passenger_count,trip_distance,amount,service_type
1273176,2023-03-10 12:32:30,2023-03-10 12:34:08,236.0,236.0,1.0,1.0,0.41,11.76,yellow
2005823,2023-04-25 10:46:38,2023-04-25 11:02:49,186.0,230.0,1.0,1.0,1.11,26.46,yellow
3899427,2023-07-27 00:51:26,2023-07-27 00:57:14,246.0,234.0,1.0,1.0,0.89,12.90,yellow
4878070,2023-08-01 15:04:45,2023-08-01 15:22:38,239.0,162.0,1.0,1.0,1.84,31.50,yellow
6075818,2023-10-16 12:46:38,2023-10-16 13:03:00,163.0,234.0,1.0,3.0,1.77,24.60,yellow
...,...,...,...,...,...,...,...,...,...
1500221,2023-03-23 04:47:16,2023-03-23 05:08:32,161.0,138.0,1.0,1.0,12.77,78.95,yellow
1516783,2023-03-21 20:14:44,2023-03-21 20:21:16,75.0,74.0,1.0,1.0,1.40,16.66,yellow
7341888,2023-12-14 22:23:31,2023-12-14 22:30:19,142.0,161.0,1.0,2.0,1.11,17.68,yellow
2686525,2023-05-15 17:30:34,2023-05-15 17:53:47,113.0,163.0,1.0,1.0,2.57,27.00,yellow


Concatenando ambos DF Yellow y Green

In [27]:
# Concatenamos los DataFrames de yellow y green taxis
taxis = pd.concat([filas_yellow_seleccionadas, filas_green_seleccionadas], ignore_index=True)
taxis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315534 entries, 0 to 315533
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   pickup_datetime   315534 non-null  datetime64[ns]
 1   dropoff_datetime  315534 non-null  datetime64[ns]
 2   PULocationID      315534 non-null  float64       
 3   DOLocationID      315534 non-null  float64       
 4   RatecodeID        315534 non-null  float64       
 5   passenger_count   315534 non-null  float64       
 6   trip_distance     315534 non-null  float64       
 7   amount            315534 non-null  float64       
 8   service_type      315534 non-null  object        
dtypes: datetime64[ns](2), float64(6), object(1)
memory usage: 21.7+ MB


In [28]:
taxis

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,RatecodeID,passenger_count,trip_distance,amount,service_type
0,2023-03-10 12:32:30,2023-03-10 12:34:08,236.0,236.0,1.0,1.0,0.41,11.76,yellow
1,2023-04-25 10:46:38,2023-04-25 11:02:49,186.0,230.0,1.0,1.0,1.11,26.46,yellow
2,2023-07-27 00:51:26,2023-07-27 00:57:14,246.0,234.0,1.0,1.0,0.89,12.90,yellow
3,2023-08-01 15:04:45,2023-08-01 15:22:38,239.0,162.0,1.0,1.0,1.84,31.50,yellow
4,2023-10-16 12:46:38,2023-10-16 13:03:00,163.0,234.0,1.0,3.0,1.77,24.60,yellow
...,...,...,...,...,...,...,...,...,...
315529,2023-08-13 08:44:00,2023-08-13 08:50:11,129.0,129.0,1.0,1.0,2.05,19.34,green
315530,2023-05-28 13:06:28,2023-05-28 13:18:42,74.0,151.0,1.0,1.0,2.20,24.00,green
315531,2023-11-21 19:05:28,2023-11-21 19:09:29,75.0,262.0,1.0,1.0,0.80,13.25,green
315532,2023-11-20 14:57:58,2023-11-20 15:07:13,75.0,41.0,1.0,2.0,0.97,11.50,green


In [29]:
taxis.PULocationID.unique()

array([236., 186., 246., 239., 163.,  68., 230.,  43., 234., 170.,  74.,
       144., 142., 114., 132., 161., 141., 224., 237., 100., 238., 231.,
       264., 233., 162., 138., 249.,  24., 262.,  50., 164., 143., 125.,
        13., 140., 229., 107., 263., 165., 113.,  79., 148.,  70., 179.,
        89.,  48., 137., 151.,  75., 261., 211.,  90.,  65.,  83., 166.,
       152., 158.,   7.,  17.,  93.,  95., 123.,  45., 244.,  87., 209.,
       134., 127.,  91.,  35., 146., 169.,  88., 168., 121.,  55.,  41.,
        51.,  25., 208., 232.,  97.,  33.,  49., 101.,  42., 129.,   4.,
       260.,  78.,  37., 116., 112.,  29., 182., 226.,  20.,   1.,  10.,
       130., 215.,  66.,  71.,  12.,  92., 189., 145., 193., 216., 247.,
        39., 242., 195., 227., 223.,  52., 220.,  76., 265., 256.,  85.,
        61.,  72., 213.,  82.,  40.,  28., 222.,  80., 255.,  56., 139.,
       124., 157., 225.,  69.,   9.,  14., 174., 210., 235., 133., 181.,
       218., 159., 219.,  62., 147.,  26., 212., 19

In [30]:
# Función que nos devuelve el nombre del Borough en 2 nuevas columnas llamadas pickup_borough y dropoff_borough
def get_borough(location_id):

    # Agrupamos los PULocationID y DOLocationID de acuerdo a su Borough correspondiente
    zones = {
        'Manhattan' : [4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 103, 103, 103, 107, 113, 114, 116, 120, 125,
                      127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211,
                      224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263],


        'Brooklyn' : [11, 14, 17, 21, 22, 25, 26, 29, 33, 34, 35, 36, 37, 39, 40, 49, 52, 54, 55, 61, 62, 63, 65, 66, 67, 71, 72, 76, 77, 80,
                     85, 89, 91, 97, 106, 108, 111, 112, 123, 133, 149, 150, 154, 155, 165, 177, 178, 181, 188, 189, 190, 195, 210, 217, 222,
                     225, 227, 228, 255, 256, 257],


        'Queens' : [2, 7, 8, 9, 10, 15, 16, 19, 27, 28, 30, 38, 53, 56, 56, 64, 70, 73, 82, 83, 86, 92, 93, 95, 96, 98, 101, 102, 117, 121,
                   122, 124, 129, 130, 131, 132, 134, 135, 138, 139, 145, 146, 157, 160, 171, 173, 175, 179, 180, 191, 192, 193, 196, 197,
                   198, 201, 203, 205, 207, 215, 216, 218, 219, 223, 226, 252, 253, 258, 260],


        'Bronx' : [3, 18, 20, 31, 32, 46, 47, 51, 58, 59, 60, 69, 78, 81, 94, 119, 126, 136, 147, 159, 167, 168, 169, 174, 182, 183, 184,
                   185, 199, 200, 208, 212, 213, 220, 235, 240, 241, 242, 247, 248, 250, 254, 259],
    
        'Staten Island' : [5, 6, 23, 44, 84, 99, 109, 110, 115, 118, 156, 172, 176, 187, 204, 206, 214, 221, 245, 251],
    
    
        'EWR' : [1]}
    
    for borough, ids in zones.items():
        if location_id in ids:
            return borough
    return 'Unknown'

In [31]:
# Aplicamos la función para crear nuevas columnas
taxis['pickup_borough'] = taxis['PULocationID'].apply(get_borough)
taxis['dropoff_borough'] = taxis['DOLocationID'].apply(get_borough)

In [32]:
taxis.pickup_borough.value_counts()

pickup_borough
Manhattan        277768
Queens            32270
Unknown            2609
Brooklyn           2330
Bronx               536
EWR                  12
Staten Island         9
Name: count, dtype: int64

In [33]:
taxis.dropoff_borough.value_counts()

dropoff_borough
Manhattan        278673
Queens            17223
Brooklyn          12669
Unknown            3988
Bronx              1998
EWR                 902
Staten Island        81
Name: count, dtype: int64

In [34]:
# Eliminamos todas las filas donde pickup_borough o dropoff_borough sea igual a 'Unknown'
taxis = taxis[(taxis['pickup_borough'] != 'Unknown') & (taxis['dropoff_borough'] != 'Unknown')]
taxis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 310971 entries, 0 to 315533
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   pickup_datetime   310971 non-null  datetime64[ns]
 1   dropoff_datetime  310971 non-null  datetime64[ns]
 2   PULocationID      310971 non-null  float64       
 3   DOLocationID      310971 non-null  float64       
 4   RatecodeID        310971 non-null  float64       
 5   passenger_count   310971 non-null  float64       
 6   trip_distance     310971 non-null  float64       
 7   amount            310971 non-null  float64       
 8   service_type      310971 non-null  object        
 9   pickup_borough    310971 non-null  object        
 10  dropoff_borough   310971 non-null  object        
dtypes: datetime64[ns](2), float64(6), object(3)
memory usage: 28.5+ MB


In [35]:
# Cambiamos el tipo de dato de 'passenger_count' a int
taxis['passenger_count'] = taxis['passenger_count'].astype(int)
taxis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 310971 entries, 0 to 315533
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   pickup_datetime   310971 non-null  datetime64[ns]
 1   dropoff_datetime  310971 non-null  datetime64[ns]
 2   PULocationID      310971 non-null  float64       
 3   DOLocationID      310971 non-null  float64       
 4   RatecodeID        310971 non-null  float64       
 5   passenger_count   310971 non-null  int32         
 6   trip_distance     310971 non-null  float64       
 7   amount            310971 non-null  float64       
 8   service_type      310971 non-null  object        
 9   pickup_borough    310971 non-null  object        
 10  dropoff_borough   310971 non-null  object        
dtypes: datetime64[ns](2), float64(5), int32(1), object(3)
memory usage: 27.3+ MB


C:\Users\Pc\AppData\Local\Temp\ipykernel_8356\3230979548.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxis['passenger_count'] = taxis['passenger_count'].astype(int)


In [36]:
taxisdf = taxis[['pickup_datetime', 'dropoff_datetime', 'RatecodeID', 'passenger_count', 'trip_distance', 'amount', 'service_type', 'pickup_borough', 'dropoff_borough']]
taxisdf

,pickup_datetime,dropoff_datetime,RatecodeID,passenger_count,trip_distance,amount,service_type,pickup_borough,dropoff_borough
0,2023-03-10 12:32:30,2023-03-10 12:34:08,1.0,1,0.41,11.76,yellow,Manhattan,Manhattan
1,2023-04-25 10:46:38,2023-04-25 11:02:49,1.0,1,1.11,26.46,yellow,Manhattan,Manhattan
2,2023-07-27 00:51:26,2023-07-27 00:57:14,1.0,1,0.89,12.90,yellow,Manhattan,Manhattan
3,2023-08-01 15:04:45,2023-08-01 15:22:38,1.0,1,1.84,31.50,yellow,Manhattan,Manhattan
4,2023-10-16 12:46:38,2023-10-16 13:03:00,1.0,3,1.77,24.60,yellow,Manhattan,Manhattan
...,...,...,...,...,...,...,...,...,...
315529,2023-08-13 08:44:00,2023-08-13 08:50:11,1.0,1,2.05,19.34,green,Queens,Queens
315530,2023-05-28 13:06:28,2023-05-28 13:18:42,1.0,1,2.20,24.00,green,Manhattan,Manhattan
315531,2023-11-21 19:05:28,2023-11-21 19:09:29,1.0,1,0.80,13.25,green,Manhattan,Manhattan
315532,2023-11-20 14:57:58,2023-11-20 15:07:13,1.0,2,0.97,11.50,green,Manhattan,Manhattan


In [37]:
# Separar pickup_datetime en fecha y hora
taxisdf['pickup_date'] = pd.to_datetime(taxisdf['pickup_datetime']).dt.date
taxisdf['pickup_time'] = pd.to_datetime(taxisdf['pickup_datetime']).dt.time

# Separar dropoff_datetime en fecha y hora
taxisdf['dropoff_date'] = pd.to_datetime(taxisdf['dropoff_datetime']).dt.date
taxisdf['dropoff_time'] = pd.to_datetime(taxisdf['dropoff_datetime']).dt.time

# Eliminar columnas originales pickup_datetime y dropoff_datetime
taxisdf.drop(['pickup_datetime', 'dropoff_datetime'], axis=1, inplace=True)

C:\Users\Pc\AppData\Local\Temp\ipykernel_8356\2078944233.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxisdf['pickup_date'] = pd.to_datetime(taxisdf['pickup_datetime']).dt.date


In [38]:
# Assuming 'pickup_date' is the name of the column containing date strings
taxisdf['pickup_date'] = pd.to_datetime(taxisdf['pickup_date'])
taxisdf['dropoff_date'] = pd.to_datetime(taxisdf['dropoff_date'])


In [39]:
# Assuming 'pickup_time' is the name of the column containing time strings
taxisdf['pickup_time'] = pd.to_datetime(taxisdf['pickup_time'], format='%H:%M:%S')
taxisdf['dropoff_time'] = pd.to_datetime(taxisdf['dropoff_time'], format='%H:%M:%S')

In [40]:
taxisdf

,RatecodeID,passenger_count,trip_distance,amount,service_type,pickup_borough,dropoff_borough,pickup_date,pickup_time,dropoff_date,dropoff_time
0,1.0,1,0.41,11.76,yellow,Manhattan,Manhattan,2023-03-10,1900-01-01 12:32:30,2023-03-10,1900-01-01 12:34:08
1,1.0,1,1.11,26.46,yellow,Manhattan,Manhattan,2023-04-25,1900-01-01 10:46:38,2023-04-25,1900-01-01 11:02:49
2,1.0,1,0.89,12.90,yellow,Manhattan,Manhattan,2023-07-27,1900-01-01 00:51:26,2023-07-27,1900-01-01 00:57:14
3,1.0,1,1.84,31.50,yellow,Manhattan,Manhattan,2023-08-01,1900-01-01 15:04:45,2023-08-01,1900-01-01 15:22:38
4,1.0,3,1.77,24.60,yellow,Manhattan,Manhattan,2023-10-16,1900-01-01 12:46:38,2023-10-16,1900-01-01 13:03:00
...,...,...,...,...,...,...,...,...,...,...,...
315529,1.0,1,2.05,19.34,green,Queens,Queens,2023-08-13,1900-01-01 08:44:00,2023-08-13,1900-01-01 08:50:11
315530,1.0,1,2.20,24.00,green,Manhattan,Manhattan,2023-05-28,1900-01-01 13:06:28,2023-05-28,1900-01-01 13:18:42
315531,1.0,1,0.80,13.25,green,Manhattan,Manhattan,2023-11-21,1900-01-01 19:05:28,2023-11-21,1900-01-01 19:09:29
315532,1.0,2,0.97,11.50,green,Manhattan,Manhattan,2023-11-20,1900-01-01 14:57:58,2023-11-20,1900-01-01 15:07:13


In [41]:
df = taxisdf.copy()
df

,RatecodeID,passenger_count,trip_distance,amount,service_type,pickup_borough,dropoff_borough,pickup_date,pickup_time,dropoff_date,dropoff_time
0,1.0,1,0.41,11.76,yellow,Manhattan,Manhattan,2023-03-10,1900-01-01 12:32:30,2023-03-10,1900-01-01 12:34:08
1,1.0,1,1.11,26.46,yellow,Manhattan,Manhattan,2023-04-25,1900-01-01 10:46:38,2023-04-25,1900-01-01 11:02:49
2,1.0,1,0.89,12.90,yellow,Manhattan,Manhattan,2023-07-27,1900-01-01 00:51:26,2023-07-27,1900-01-01 00:57:14
3,1.0,1,1.84,31.50,yellow,Manhattan,Manhattan,2023-08-01,1900-01-01 15:04:45,2023-08-01,1900-01-01 15:22:38
4,1.0,3,1.77,24.60,yellow,Manhattan,Manhattan,2023-10-16,1900-01-01 12:46:38,2023-10-16,1900-01-01 13:03:00
...,...,...,...,...,...,...,...,...,...,...,...
315529,1.0,1,2.05,19.34,green,Queens,Queens,2023-08-13,1900-01-01 08:44:00,2023-08-13,1900-01-01 08:50:11
315530,1.0,1,2.20,24.00,green,Manhattan,Manhattan,2023-05-28,1900-01-01 13:06:28,2023-05-28,1900-01-01 13:18:42
315531,1.0,1,0.80,13.25,green,Manhattan,Manhattan,2023-11-21,1900-01-01 19:05:28,2023-11-21,1900-01-01 19:09:29
315532,1.0,2,0.97,11.50,green,Manhattan,Manhattan,2023-11-20,1900-01-01 14:57:58,2023-11-20,1900-01-01 15:07:13


In [42]:
df_reset = df.reset_index(drop=True)
df_reset

,RatecodeID,passenger_count,trip_distance,amount,service_type,pickup_borough,dropoff_borough,pickup_date,pickup_time,dropoff_date,dropoff_time
0,1.0,1,0.41,11.76,yellow,Manhattan,Manhattan,2023-03-10,1900-01-01 12:32:30,2023-03-10,1900-01-01 12:34:08
1,1.0,1,1.11,26.46,yellow,Manhattan,Manhattan,2023-04-25,1900-01-01 10:46:38,2023-04-25,1900-01-01 11:02:49
2,1.0,1,0.89,12.90,yellow,Manhattan,Manhattan,2023-07-27,1900-01-01 00:51:26,2023-07-27,1900-01-01 00:57:14
3,1.0,1,1.84,31.50,yellow,Manhattan,Manhattan,2023-08-01,1900-01-01 15:04:45,2023-08-01,1900-01-01 15:22:38
4,1.0,3,1.77,24.60,yellow,Manhattan,Manhattan,2023-10-16,1900-01-01 12:46:38,2023-10-16,1900-01-01 13:03:00
...,...,...,...,...,...,...,...,...,...,...,...
310966,1.0,1,2.05,19.34,green,Queens,Queens,2023-08-13,1900-01-01 08:44:00,2023-08-13,1900-01-01 08:50:11
310967,1.0,1,2.20,24.00,green,Manhattan,Manhattan,2023-05-28,1900-01-01 13:06:28,2023-05-28,1900-01-01 13:18:42
310968,1.0,1,0.80,13.25,green,Manhattan,Manhattan,2023-11-21,1900-01-01 19:05:28,2023-11-21,1900-01-01 19:09:29
310969,1.0,2,0.97,11.50,green,Manhattan,Manhattan,2023-11-20,1900-01-01 14:57:58,2023-11-20,1900-01-01 15:07:13


Guardamos el Dataset de TaxisNYC

In [43]:
df_reset.to_parquet("../0-DataSets/3-DatosScrapingConETL/taxis_2023.parquet")